# EDA on Crop Recommendation Dataset

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import kaleido

In [2]:
df=pd.read_csv('Crop_recommendation.csv')
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [3]:
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB


N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64

In [4]:
df.duplicated().sum()

np.int64(0)

In [5]:
df["label"].unique()

array(['rice', 'maize', 'chickpea', 'kidneybeans', 'pigeonpeas',
       'mothbeans', 'mungbean', 'blackgram', 'lentil', 'pomegranate',
       'banana', 'mango', 'grapes', 'watermelon', 'muskmelon', 'apple',
       'orange', 'papaya', 'coconut', 'cotton', 'jute', 'coffee'],
      dtype=object)

In [6]:
avg_npk = df.groupby("label")[["N","P","K"]].mean().reset_index()
melted = avg_npk.melt(id_vars="label", var_name="Nutrient", value_name="Value")

fig = px.bar(melted, x="label", y="Value", color="Nutrient",
             barmode="group", color_discrete_sequence=px.colors.qualitative.Vivid_r)

fig.update_layout(title="<b>Average N, P, K per Crop</b>",
                  xaxis_title="Crop", yaxis_title="Average Value",)
fig.show()
fig.write_image("plot2.png")

In [7]:
npk_data = df[["N","P","K"]].melt(var_name="Nutrient", value_name="Value")

fig = px.box(npk_data, x="Nutrient", y="Value", color="Nutrient",
             color_discrete_map={"N": "green", "P": "navy", "K": "crimson"})

fig.update_layout(
    title="<b>Distribution of N, P, K Nutrients</b>",
    yaxis_title="Value", xaxis_title="Nutrient Type"
)
fig.show()
fig.write_image("plot3.png")

In [8]:
fig = px.violin(df, x="label", y="rainfall", color="label", box=True, points="outliers",
                color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(title="<b>Rainfall Distribution per Crop</b>",
                  xaxis_title="Crop", yaxis_title="Rainfall")
fig.show()
fig.write_image("plot4.png")

In [9]:
avg = df.groupby("label").mean().reset_index()
features = ["N","P","K","temperature","humidity","ph","rainfall"]
fig = go.Figure()
for f in features:
    fig.add_trace(go.Scatter(
        x=avg["label"], y=avg[f], mode='lines', stackgroup='one', name=f
    ))
fig.update_layout(
    title="<b>Mean Feature Levels per Crop</b>", xaxis_title="Crop",
    yaxis_title="Mean Values", hovermode="x unified"
)
fig.show()
fig.write_image("plot7.png")

In [10]:
df["NPK_balance"] = (df["N"] + df["P"] + df["K"]) / 3

df["moisture_index"] = df["humidity"] * df["rainfall"]

def ph_category(p):
    if p < 5.5:
        return "Acidic"
    elif p <= 7:
        return "Neutral"
    else:
        return "Alkaline"

df["pH_category"] = df["ph"].apply(ph_category)

In [11]:
avg_npk = df.groupby("label")["NPK_balance"].mean().reset_index()
avg_npk = avg_npk.sort_values("NPK_balance", ascending=False)

fig = px.bar(
    avg_npk, x="label", y="NPK_balance",
    title="Average Nutrient Score per Crop",
    color="NPK_balance", color_continuous_scale="Oranges"
)
fig.update_layout(
    xaxis_title="Crop", yaxis_title="NPK Balance Score", showlegend=False
)
fig.show()

In [12]:
fig = px.scatter(
    df, x="temperature", y="moisture_index", color="label",
    title="<b>Moisture vs Temperature</b>", opacity=0.7,
    hover_data=["N","P","K","humidity","rainfall"]
)

fig.update_layout(
    xaxis_title="Temperature (°C)", yaxis_title="Moisture Index"
)
fig.show()
fig.write_image("plot5.png")

In [13]:
ph_counts = df["pH_category"].value_counts().reset_index()
ph_counts.columns = ["pH_category", "count"]
fig = px.pie(
    ph_counts, names="pH_category", values="count", hole=0.45,
    title="Soil pH Category Distribution", color="pH_category",
    color_discrete_map={"Acidic": "#EF553B", "Neutral": "#00CC96", "Alkaline": "#636EFA"}
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()
fig.write_image("plot8.png")

In [14]:
import plotly.figure_factory as ff

corr = df.drop(["label", "pH_category"], axis=1).corr().round(2)
fig = ff.create_annotated_heatmap(
    z=corr.values, x=list(corr.columns), y=list(corr.columns), colorscale="Viridis"
)
fig.update_layout(title="<b>Correlation Heatmap</b>")
fig.show()
fig.write_image("plot10.png")